# In this assignment students need to predict whether a person makes over 50K per year or not from classic adult dataset using XGBoost. The description of the dataset is as follows:

#Data Set Information:
Extraction was done by Barry Becker from the 1994 Census database. A set of
reasonably clean records was extracted using the following conditions: ((AAGE>16) &&
(AGI>100) && (AFNLWGT>1)&& (HRSWK>0))

In [4]:
#imposting the libraries

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import confusion_matrix

# loading the data
train_set =pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None)

test_set =pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test', skiprows = 1, header = None)

#naming the columns
col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
'occupation','relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week',
'native_country', 'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

#droping the missing values
train_nomissing = train_set.replace(' ?', np.nan).dropna()
test_nomissing = test_set.replace(' ?', np.nan).dropna()

#romoving dot from Wage_class of test data
test_nomissing['wage_class'] = test_nomissing.wage_class.replace({' <=50K.': ' <=50K', ' >50K.':' >50K'})

#joining the train data and test data vertically
combined_set = pd.concat([train_nomissing, test_nomissing], axis = 0) 

# converting the categorical values to numbers
for feature in combined_set.columns: # Loop through all columns in the dataframe
    if combined_set[feature].dtype == 'object': # Only apply for columns with categorical strings
        combined_set[feature] = pd.Categorical(combined_set[feature]).codes # Replace strings with an integer

#separating the train data and test data
final_train = combined_set[:train_nomissing.shape[0]] # Up to the last initial training set row
final_test = combined_set[train_nomissing.shape[0]:] # Past the last initial training set row


y_train = final_train.pop('wage_class')
y_test = final_test.pop('wage_class')

params = {
    'objective': 'binary:logistic',
    'max_depth': 6,
    'learning_rate': 1.0,
    'silent': 1.0,
    'n_estimators': 10
}

# creating the xgboost classifier
model = XGBClassifier(**params)
model.fit(final_train,y_train)

#predicting the values
y_pred= model.predict(final_test)

#calculating correct matched values
correct = 0
for i in range(len(y_pred)):
    if (y_test[1] == y_pred[i]):
        correct += 1

print('predicted correctly: {0}/{1}'.format( correct,len(y_pred)))


# calculating the accuracy       
acc = accuracy_score(y_test,y_pred)
print("accuracy_score:",end=" ")
print(acc)


print("confusion matrix:")
print(confusion_matrix(y_test,y_pred))

C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


predicted correctly: 11868/15060
accuracy_score: 0.8638778220451527
confusion matrix:
[[10589   771]
 [ 1279  2421]]
